<a href="https://colab.research.google.com/github/realBagher/sentiment_analysis_food_classifcation/blob/main/sent_inference_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import numpy as np
import pandas as pd
import re
import os
import zipfile

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split, KFold

import keras
import tensorflow as tf
import tensorflow_datasets as tfds
# import tensorflow_text
from tensorflow.keras import Sequential, regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, GlobalAveragePooling1D, LSTM, Bidirectional
from tensorflow.python.client import device_lib
# from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
import plotly.graph_objs as go
from plotly.subplots import make_subplots
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
from keras.layers import Bidirectional
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix,classification_report
import spacy,string
import tqdm
tqdm.tqdm.pandas()

In [ ]:
import pandas as pd
import gzip
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
import torch
import numpy as np

In [8]:
df = pd.read_csv("/content/drive/MyDrive/project2/train_data.csv")
df= df.sample(frac=1)[['reviewText','summary','overall']]
df.shape

<ipython-input-8-bea09e65e39a>:1: DtypeWarning:

Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.



(838944, 3)

In [9]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
print(stopwords.words('english'))
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df['summary']=df['summary'].astype(str)
df['review_clean'] = df['reviewText'].progress_apply(clean_text)
df['summary_clean'] = df['summary'].progress_apply(clean_text)
df['total_text']=df['summary']+'.'+df['review_clean']

100%|██████████| 838944/838944 [00:05<00:00, 165757.68it/s]


In [12]:
new_df= df

In [13]:
new_df.dropna(subset=['reviewText', 'overall'], inplace=True)
new_df['overall'] = new_df['overall'] - 1

In [14]:
df_test = new_df[['total_text','overall']].iloc[:50000, :].reset_index(drop=True)
df_test.rename({'total_text':'text','overall':'label'},axis=1,inplace=True)
df_train = new_df[['total_text','overall']].iloc[50000:, :].reset_index(drop=True)
df_train.rename({'total_text':'text','overall':'label'},axis=1,inplace=True)

In [50]:
# train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test[:1000])

In [17]:
!unzip -o /content/drive/MyDrive/project2/model_large_distilbert_3epoch.zip -d  final_Model

Archive:  /content/drive/MyDrive/project2/model_large_distilbert_3epoch.zip
   creating: final_Model/final_Model/
  inflating: final_Model/final_Model/model.safetensors  
  inflating: final_Model/final_Model/config.json  
  inflating: final_Model/final_Model/training_args.bin  


In [18]:
from transformers import pipeline
from datasets import load_dataset
from evaluate import evaluator
import evaluate

In [43]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [45]:
classifier = pipeline("text-classification", model='/content/final_Model/final_Model', tokenizer=tokenizer,truncation=True,device=-1,
                      batch_size=5)

In [51]:
from transformers.pipelines.pt_utils import KeyDataset
labels =[]
for out in tqdm.tqdm(classifier(KeyDataset(test_dataset, "text"))):
    if out['label']=='LABEL_0':
        labels.append(0)
    elif out['label']=='LABEL_1':
        labels.append(1)
    elif out['label']=='LABEL_2':
        labels.append(2)
    elif out['label']=='LABEL_3':
        labels.append(3)
    else:
        labels.append(4)

1000it [06:28,  2.57it/s]


In [53]:
from sklearn.metrics import f1_score
f1 = f1_score(test_dataset['label'], labels, average='weighted')
f1

0.7288246572659262

In [54]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_dataset['label'], labels)
accuracy

0.744

In [23]:
!pip install gradio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━

In [55]:
import gradio as gr
#setup

model_path = "/content/final_Model/final_Model"
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained(model_path)

def preprocess(text):
    # Preprocess text (username and link placeholders)
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def sentiment_analysis(text):
    text = clean_text(text)

    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    # Make a prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class probabilities
    scores = torch.softmax(outputs.logits, dim=1).tolist()[0]

    # Map the scores to labels
    labels = ['Absolutely negative','Almost negative', 'Neutral', 'Almost positive','Absolutely positive']
    scores_dict = {label: score for label, score in zip(labels, scores)}

    return scores_dict

title = "Sentiment Analysis Application\n\n\nThis application assesses if a amazon review product realating to 'Absolutely positive,Almost positive,neutral,Almost negative,Absolutely negative' sentiments"

demo = gr.Interface(
    fn=sentiment_analysis,
    inputs=gr.Textbox(placeholder="Write your tweet here..."),
    outputs=gr.Label(num_top_classes=5),
    examples=[["This toaster is a lifesaver! It toasts bread evenly and quickly to my desired level of crispness. No more burnt mornings! The sleek design looks great on my countertop too."],
     ["The sound quality on these headphones is subpar. The bass is weak and the overall audio feels muffled. Additionally, the noise cancellation feature barely works. Disappointed with this purchase."],
      ["The quality of this dress shirt is terrible. Feels like it will fall apart after one wear. Disappointed considering the price."],
       ["This frozen pizza is a decent option for a quick and easy meal. The crust is crispy and the toppings are flavorful. Not gourmet, but definitely satisfies a pizza craving."],
        ["Unfortunately, this shampoo didn't work well for my hair type. It made my hair feel dry and frizzy. Might be a good option for people with oily hair, but not for those with dry hair."],
              ["These jeans are terrible! The sizing is way off, and the material feels cheap and uncomfortable. They also ripped after just a few wears. Very disappointed with the quality"],
              ["hese coffee pods taste burnt and bitter. They're also weak and don't deliver a strong coffee flavor. Definitely not worth the price. Avoid these!"],
              ["This face moisturizer is decent. It provides a good amount of hydration without feeling greasy. However, I haven't noticed any significant improvements in my skin after using it for a while. Maybe I need to use it longer to see results"],
              ["This self-help book has been incredibly helpful for me during a challenging time. The author's insights and practical tips have been invaluable, and I've already started to see positive changes in my life. Highly recommend for anyone looking for self-improvement."]],
     title=title
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e9ce330b2b642fbbed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
